# ML05 Support Vector Machine (SVM)

In [1]:
# Install cuml

!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
Installing RAPIDS remaining 24.4.* libraries
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com

        ***********************************************************************
        The pip install of RAPIDS is complete.
        
        Please do not run any further installation from the conda based installation methods, as they may cause issues!
        
        Please ensure that you're pulling from the git repo to remain updated with the latest working install scripts.

        Troubleshooting:
            - If there is an installation failure, please check back on RAPIDSAI owned templates/notebooks to see how to update your personal files. 
            - If an installation failure persists when using the latest script, please make an issue on https://github.com/rapidsai-community/rapidsai-csp-utils
        *****************************************************************

In [2]:
#Download MNIST dataset (via Keras)

import numpy as np
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

#Normalize
x_train = np.array([np.ravel(x) for x in x_train]).astype(np.float32) / 225.0
x_test = np.array([np.ravel(x) for x in x_test]).astype(np.float32) / 225.0

In [3]:
import time

#cuml
from cuml.model_selection import train_test_split
from cuml.svm import SVC, SVR


#Train
t0 = time.time()
svc = SVC(C=0.5, kernel="linear", degree=2)
svc.fit(x_train,y_train)
train_time = time.time() - t0

print(f'Train time: {train_time}')


#Predict
t0 = time.time()
pred = svc.predict(x_test)
pred_time = time.time() - t0

print(f'Predict time: {pred_time}')

[W] [09:12:52.909676] SVC with the linear kernel can be much faster using the specialized solver provided by LinearSVC. Consider switching to LinearSVC if tranining takes too long.
Train time: 9.888054609298706
Predict time: 1.1430847644805908


In [4]:
#Scores
from sklearn.metrics import classification_report

print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97       980
           1       0.98      0.99      0.98      1135
           2       0.94      0.93      0.93      1032
           3       0.91      0.94      0.93      1010
           4       0.94      0.96      0.95       982
           5       0.91      0.90      0.91       892
           6       0.96      0.96      0.96       958
           7       0.95      0.93      0.94      1028
           8       0.91      0.91      0.91       974
           9       0.94      0.92      0.93      1009

    accuracy                           0.94     10000
   macro avg       0.94      0.94      0.94     10000
weighted avg       0.94      0.94      0.94     10000



In [6]:
#Save model

import pickle

with open('svm_mnist_model.pkl', 'wb') as f:
    pickle.dump(svc, f)